In [12]:
import faiss
from uuid import uuid4
from langchain_openai import OpenAIEmbeddings
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

import requests
from bs4 import BeautifulSoup

In [2]:
def scrap_meli_url(url: str) -> str:
    # 1. Descargar HTML
    headers = {
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64)"
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    # 2. Extraer el H1
    h1 = soup.find("h1", class_="cx-peach-content__title")
    titulo = h1.get_text(strip=True) if h1 else None

    # 3. Extraer el contenido debajo del H1
    # Mercado Libre suele ponerlo en un div con esta clase
    contenido_div = soup.find("div", class_="cx-peach-content__data")

    if contenido_div:
        # Evitar saltos y espacios raros
        contenido = "\n".join(p.get_text(strip=True) for p in contenido_div.find_all("p"))
    else:
        contenido = None

    string_to_return = f"""
    PREGUNTA:\n{titulo}
    
    RESPUESTA:\n{contenido}
    """
    return string_to_return

In [3]:
url_list_from_meli_help_buyings_page = [
    "https://www.mercadolibre.com.mx/ayuda/Como-elegir-a-que-vendedor-comprarle_640", # Cómo elegir a qué vendedor comprarle
    "https://www.mercadolibre.com.mx/ayuda/Como-pagar-tu-compra_923", # Cómo pagar tu compra 
    "https://www.mercadolibre.com.mx/ayuda/26594", # Cómo usar cupones
    "https://www.mercadolibre.com.mx/ayuda/Como-protegemos-a-compradores_925", # Cómo protegemos a los compradores
    "https://www.mercadolibre.com.mx/ayuda/23867", # Qué hacer en caso de desconocer un cargo en la tarjeta bancaria
    "https://www.mercadolibre.com.mx/ayuda/4603", # Beneficio de envío gratis
    "https://www.mercadolibre.com.mx/ayuda/30191", # Contracargo por desconocer un cobro
    "https://www.mercadolibre.com.mx/ayuda/30214", # Cómo pagar una deuda
    "https://www.mercadolibre.com.mx/ayuda/Recibir-el-producto-en-mi-casa_3228", # Cómo recibir el producto en mi casa
    "https://www.mercadolibre.com.mx/ayuda/26594", # Cómo usar cupones en mi compra
]

In [4]:
scrapped_texts = []
for _url in url_list_from_meli_help_buyings_page:
    scrapped_texts.append(
        scrap_meli_url(
            url=_url
        )
    )

In [5]:
for _scrapped_text in scrapped_texts:
    print(_scrapped_text, "\n\n")


    PREGUNTA:
Cómo elegir a qué vendedor comprarle

    RESPUESTA:
Si te gustó un producto y tienes dudas sobre el vendedor, te damos algunos consejos para que tengas en cuenta:

Fíjate el color como vendedor
Los vendedores que tienen color verde son aquellos que ofrecen un buen servicio a sus compradores. Si es gris, es un vendedor sin ventas o con pocas, pero no quiere decir que sea malo.
Si hay medalla, mucho mejor
Es un MercadoLíder, un vendedor avalado por la cantidad de ventas que hizo, la buena experiencia que brinda a sus compradores y el cumplimiento de normas fiscales.

Si quieres comprar un auto o un inmueble, siempre es bueno llamar al anunciante para coordinar una visitay sacarte todas las dudas sobre lo que quieres comprar.
Si después de leer toda la publicación aún tienes dudas,pregúntale al vendedor todo lo que necesites saber. Por ejemplo:si necesitas la facturade tu compra, hazle una pregunta yasegúrate de que te la envíe.
     



    PREGUNTA:
Cómo pagar tu compra


# Generando el FAISS Vectorial Database

## Generar los Document para cargar en la vectorial


In [10]:
from langchain_core.documents import Document as LangchainDocument

In [11]:
all_langchain_documets = []
for _scrapped_text in scrapped_texts:
    all_langchain_documets.append(
        LangchainDocument(
            page_content=_scrapped_text,
            metadata={"source": "Mercado Libre Website sección de Ayuda"}
        )   
    )

## Crear la base de datos vectorial en memoria

In [6]:
OPENAI_EMBEDDINGS_MODEL_DIMENSION = 1024

In [7]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large", dimensions=OPENAI_EMBEDDINGS_MODEL_DIMENSION)

In [8]:
index = faiss.IndexFlatL2(OPENAI_EMBEDDINGS_MODEL_DIMENSION)

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

## Cargar los langchain documents en la base de datos vectorial

In [13]:
uuids = [str(uuid4()) for _ in range(len(all_langchain_documets))]
vector_store.add_documents(documents=all_langchain_documets, ids=uuids)

['a5bd145d-0203-485d-99ce-f887fcdafad3',
 'af34f73b-7f6d-434d-8e76-b39f3399b76e',
 '5f69c780-6442-4c51-9361-bb9aed56b879',
 'e1f30cb5-c5e0-4cb2-9d42-0655f9ab985c',
 '073a631b-fc97-4b99-8245-73c27d175bf8',
 '84716eb8-3f8f-459b-8795-4570c564db7e',
 'de47acea-669e-47c6-a607-74f66e4bd2cf',
 'a3d99eac-3f1c-45b8-8889-9bc904f7ec77',
 'b45bdde9-dbea-48ce-b8e0-d9a4f5cd48c2',
 '6c357efa-f4d0-44fa-a9c7-ec4c282a149c']

# Guardar la base de datos vectorial en el storage local#

In [14]:
vector_store.save_local(
    folder_path="../data/vector_stores/faiss/MeliFAQsIndex/",
    index_name="MeliFAQsIndex",
)

### Query vectorial

In [15]:
results = vector_store.similarity_search_with_relevance_scores(
    query="Cómo pagar un pago en meli para que este quede pagado",
    k=3
)

In [20]:
for _result in results:
    print("SCORE DE SIMILITUD DEL DOCUMENTO:\n\n", _result[1])
    print("CONTENIDO DEL DOCUMENTO:\n\n", _result[0].page_content)

SCORE DE SIMILITUD DEL DOCUMENTO:

 0.4403187
CONTENIDO DEL DOCUMENTO:

 
    PREGUNTA:
Cómo pagar tu compra

    RESPUESTA:
Pagarcon Mercado Pagoes pagar con cualquiera de estos medios. Tucompra va a estar 100% protegiday te devolveremos el dinero si el producto no es lo que esperabas.
Los apartados de vehículos también se pagan con Mercado Pago. Así protegemos tu dinero hasta que nos confirmes que ya te entregaron el vehículo.

Elige entre cualquiera de estos medios de pago deMercado Pago:
Tarjeta de crédito Mercado Pago:selecciona Tarjeta de Crédito Mercado Pago como medio de pago, elige las mensualidades o paga de contado, ¡y listo!
Conoce más
Hasta 18 mensualidades
Acreditación instantánea
Visa
Mastercard
American Express
Tarjeta de crédito
Hasta 24 mensualidades
Acreditación instantánea
Bancomer
Banamex
HSBC
Santander
Banorte
IXE
Tarjeta de débito o recargables
Al contado
Acreditación instantánea

SeleccionaMeses sin Tarjetacomo medio de pago y luego elige las mensualidades para 